In [5]:
"""
"""

import os, glob
import h5py

# File path information
maindir = '/Users/max/Library/CloudStorage/OneDrive-Personal/mcook/aspen-fire'
datadir = os.path.join(maindir,'Aim2/data/spatial/raw/VIIRS/')


In [9]:
def list_files(path, ext, recursive):
    """
    List files of a specific type in a directory or subdirectories
    """
    if recursive is True:
        return glob.glob(os.path.join(path, '**', '*{}'.format(ext)), recursive=True)
    else:
        return glob.glob(os.path.join(path, '*{}'.format(ext)), recursive=False)


def read_h5_file(fp):
    geo_objs = []
    def print_attrs(name, obj):
        print(name)
        for key, val in obj.attrs.items():
            print(f"    {key}: {val}")     
    with h5py.File(fp, 'r') as f:
        print(f'Contents of {fp}')
        f.visititems(print_attrs)
        f.visit(geo_objs.append) # stores the SDS objects
    return geo_objs

In [7]:
# Testing for one fire
testDir = os.path.join(datadir,'FIRED_3518')
# Get a list of geo files
geo_files = list_files(testDir,"*.h5",recursive=True)
print(geo_files[0])

/Users/max/Library/CloudStorage/OneDrive-Personal/mcook/aspen-fire/Aim2/data/spatial/raw/VIIRS/FIRED_3518/VJ103MODLL/VJ103MODLL.A2021243.1000.021.2021243163653.h5


In [10]:
geo_objects = read_h5_file(geo_files[0])

Contents of /Users/max/Library/CloudStorage/OneDrive-Personal/mcook/aspen-fire/Aim2/data/spatial/raw/VIIRS/FIRED_3518/VJ103MODLL/VJ103MODLL.A2021243.1000.021.2021243163653.h5
HDFEOS
    Conventions: b'CF-1.6'
    EndTime: b'2021-08-31 10:06:00.000'
    GRingPointSequenceNo: [[1 2 3 4]]
    TAI93_leapseconds: 10
    _NCProperties: b'version=1|netcdflibversion=4.4.1.1|hdf5libversion=1.8.17'
    date_created: b'2021-08-31T12:20:50Z'
    endDirection: b'Descending'
    format_version: 2
    geospatial_lat_max: 48.82603454589844
    geospatial_lat_min: 23.335987091064453
    geospatial_lat_units: b'degrees_north'
    geospatial_lon_max: -101.1566162109375
    geospatial_lon_min: -141.2718048095703
    geospatial_lon_units: b'degrees_east'
    history: b'geolocate_viirs.exe PGE501.1.AS5201.JP1VIIRS.2021-243T10:00.NA.2021-243T16:14:49.48370767.pcf'
    identifier_product_doi: b'10.5067/VIIRS/VJ103MOD.021'
    identifier_product_doi_authority: b'https://doi.org'
    instrument: b'VIIRS'
    in

In [11]:
(geo_objects)

['HDFEOS',
 'HDFEOS/ADDITIONAL',
 'HDFEOS/ADDITIONAL/FILE_ATTRIBUTES',
 'HDFEOS/SWATHS',
 'HDFEOS/SWATHS/VNP_750M_GEOLOCATION',
 'HDFEOS/SWATHS/VNP_750M_GEOLOCATION/Data Fields',
 'HDFEOS/SWATHS/VNP_750M_GEOLOCATION/Data Fields/Height',
 'HDFEOS/SWATHS/VNP_750M_GEOLOCATION/Geolocation Fields',
 'HDFEOS/SWATHS/VNP_750M_GEOLOCATION/Geolocation Fields/Latitude',
 'HDFEOS/SWATHS/VNP_750M_GEOLOCATION/Geolocation Fields/Longitude',
 'HDFEOS INFORMATION',
 'HDFEOS INFORMATION/StructMetadata.0']

In [13]:
# Get a list of .nc files
nc_files = list_files(os.path.join(maindir, 'Aim2/data/spatial/raw/VIIRS'), "*.nc", recursive=True)
# Seperate into VNP14 and VJ114, VNP03
print(len(nc_files))

vnp = [f for f in nc_files if 'VNP14' in os.path.basename(f)] # VNP14IMG
vji = [f for f in nc_files if 'VJ114' in os.path.basename(f)] # VJI14IMG

print(f'VNP14IMG: {len(vnp)}')
print(f'VJI14IMG: {len(vji)}')

313
VNP14IMG: 158
VJI14IMG: 155


In [14]:
# Library to work with netCDF files
from netCDF4 import Dataset

# Open a .nc file ("file_name")
ds = Dataset(vnp[0], 'r', format = 'NETCDF4')

print(ds)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    VNP02IMG: VNP02CCIMG.A2019225.1936.002.2022333062425.nc
    VNP03IMG: VNP03IMG.A2019225.1936.002.2021110215728.nc
    VNP02MOD: VNP02CCMOD.A2019225.1936.002.2022333062425.nc
    VNP02GDC: VNP02GDC.A2019225.1936.002.2021111141115.nc
    ProcessVersionNumber: 3.1.9
    ExecutableCreationDate: Oct 31 2023
    ExecutableCreationTime: 17:46:00
    SystemID: Linux minion20367 5.4.0-1093-fips #103-Ubuntu SMP Thu Feb 8 14:02:37 UTC 2024 x86_64
    Unagg_GRingLatitude: 51.740856,59.026741,38.287682,33.210144
    Unagg_GRingLongitude: -129.983383,-81.693024,-81.958115,-115.527855
    NorthBoundingCoordinate: 59.20688
    SouthBoundingCoordinate: 33.210144
    EastBoundingCoordinate: -81.69302
    WestBoundingCoordinate: -129.98338
    Day/Night/Both: Day
    FirePix: 63
    DayPix: 41369600
    LandPix: 39874606
    Satellite: NPP
    NightPix: 0
    DayNightFlag: Day
    WaterPix: 1494994
    MissingPix: 9

In [15]:
# Print a list of variables in file
list(ds.variables)

['FP_AdjCloud',
 'FP_AdjWater',
 'FP_MAD_DT',
 'FP_MAD_T4',
 'FP_MAD_T5',
 'FP_MeanDT',
 'FP_MeanRad13',
 'FP_MeanT4',
 'FP_MeanT5',
 'FP_Rad13',
 'FP_SolAzAng',
 'FP_SolZenAng',
 'FP_T4',
 'FP_T5',
 'FP_ViewAzAng',
 'FP_ViewZenAng',
 'FP_WinSize',
 'FP_confidence',
 'FP_day',
 'FP_latitude',
 'FP_line',
 'FP_longitude',
 'FP_power',
 'FP_sample',
 'algorithm QA',
 'fire mask']

In [21]:
ds.variables['fire mask'][:]

masked_array(
  data=[[1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        ...,
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1]],
  mask=False,
  fill_value=999999,
  dtype=uint8)

In [18]:
# Grab variable and variable metadata
fire_mask = ds.variables['fire mask'][:]
fire_mask.shape

(6464, 6400)

In [19]:
type(fire_mask)

numpy.ma.core.MaskedArray

In [20]:
fire_mask

masked_array(
  data=[[1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        ...,
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1]],
  mask=False,
  fill_value=999999,
  dtype=uint8)